# 1. Generate Task 0 BossLevel Data

In [2]:
import blosc
from collections import defaultdict
import io
import os
import pickle as pkl
from PIL import Image
from tqdm import tqdm

In [3]:
def get_pretraining_data(
    env,
    data,
    output_dir_path,
    is_train,
    split_size=100,
    split_start_id=0,
):
    output_dir = os.path.join(output_dir_path, env)
    os.makedirs(output_dir, exist_ok=True)
    
    split_id = 0
    data_split = []
    for ex_id, ex in tqdm(enumerate(data), total=len(data)):
        img = blosc.unpack_array(ex[2])[0]  # Always select the first frame
        #img_encoded = image_to_bytes(img)
        #data_split["img"].append(img[0])  # Always select the first frame
        
        #data_split["mission"].append(ex[0])

        action = ",".join(ex[6])
        #data_split["action_seq"].append(action)
        data_split.append({
            "img": img,
            "mission": ex[0],
            "label": ex[-1]
        })
        
        if (ex_id+1) % split_size == 0:
            if is_train:
                output_name = f"split_train_{split_start_id + split_id}"
            else:
                output_name = f"split_valid_{split_start_id + split_id}"
            
            output_path = os.path.join(output_dir, f"{output_name}.pkl")
            with open(output_path, "wb") as f:
                pkl.dump(data_split, f)
            data_split = []
            split_id += 1

    if data_split:
        if is_train:
            output_name = f"split_train_{split_start_id + split_id}"
        else:
            output_name = f"split_valid_{split_start_id + split_id}"

        output_path = os.path.join(output_dir, f"{output_name}.pkl")
        with open(output_path, "wb") as f:
            pkl.dump(data_split, f)
    
    return split_start_id + split_id  # used to track next starting splid_id

In [5]:
level_name = "BossLevel"

split_start_id = 0

for split_id in range(10):
    print(split_id)
    with open(f"../../data/task_0/{level_name}/Task0_{level_name}_train_{split_id}.pkl", "rb") as f:
        demos_train = pkl.load(f)
    
    split_start_id = get_pretraining_data(
        level_name, 
        demos_train, 
        output_dir_path="../../data/clip_experiment",
        is_train=True,
        split_size=500,
        split_start_id=split_start_id,
    )

0


100%|███████████████████████████████████████| 5000/5000 [05:08<00:00, 16.21it/s]


1


100%|███████████████████████████████████████| 5000/5000 [05:13<00:00, 15.96it/s]


2


 58%|██████████████████████▌                | 2891/5000 [03:30<02:33, 13.73it/s]


KeyboardInterrupt: 

In [ ]:
level_name = "BossLevel"

split_start_id = 0

for split_id in range(10):
    print(split_id)
    with open(f"../../data/task_0/{level_name}/Task0_{level_name}_valid_{split_id}.pkl", "rb") as f:
        demos_valid = pkl.load(f)
    
    split_start_id = get_pretraining_data(
        level_name, 
        demos_valid, 
        output_dir_path="../../data/clip_experiment",
        is_train=False,
        split_size=500,
        split_start_id=split_start_id,
    )